<a href="https://www.kaggle.com/code/realshaktigupta/forecasting-competition?scriptVersionId=138115413" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
import numpy as np
import pandas as pd
import os
import tensorflow as tf
import keras
import tensorflow.keras.backend as K
import datetime
from tqdm import tqdm
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import BaggingRegressor
from catboost import CatBoostRegressor
from datetime import datetime,timedelta
from holidays import CountryHoliday
import warnings 
warnings.simplefilter("ignore")
import matplotlib.pyplot as plt

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

In [2]:
#This notebooks is a work in progress and incomplete.

In [3]:
train=pd.read_csv("/kaggle/input/playground-series-s3e19/train.csv",index_col='id')
test=pd.read_csv("/kaggle/input/playground-series-s3e19/test.csv",index_col='id')

In [4]:
train.head()

,date,country,store,product,num_sold
id,,,,,
0,2017-01-01,Argentina,Kaggle Learn,Using LLMs to Improve Your Coding,63
1,2017-01-01,Argentina,Kaggle Learn,Using LLMs to Train More LLMs,66
2,2017-01-01,Argentina,Kaggle Learn,Using LLMs to Win Friends and Influence People,9
3,2017-01-01,Argentina,Kaggle Learn,Using LLMs to Win More Kaggle Competitions,59
4,2017-01-01,Argentina,Kaggle Learn,Using LLMs to Write Better,49


In [5]:
len(train['date'].unique())

1826

In [6]:
cat_features=['country','store','product']
for feature in cat_features:
    print(feature,':',train[feature].unique())

country : ['Argentina' 'Canada' 'Estonia' 'Japan' 'Spain']
store : ['Kaggle Learn' 'Kaggle Store' 'Kagglazon']
product : ['Using LLMs to Improve Your Coding' 'Using LLMs to Train More LLMs'
 'Using LLMs to Win Friends and Influence People'
 'Using LLMs to Win More Kaggle Competitions' 'Using LLMs to Write Better']


In [7]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 136950 entries, 0 to 136949
Data columns (total 5 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   date      136950 non-null  object
 1   country   136950 non-null  object
 2   store     136950 non-null  object
 3   product   136950 non-null  object
 4   num_sold  136950 non-null  int64 
dtypes: int64(1), object(4)
memory usage: 6.3+ MB


In [8]:
def datetime_features(df_temp):
    """
ref:-https://www.kaggle.com/code/chingiznurzhanov/timeseriessplit-catboost-trick
    """
    df_temp['date']=pd.to_datetime(df_temp['date'],format='%Y-%m-%d')
    df_temp['month'] = df_temp['date'].dt.month
    df_temp['day'] = df_temp['date'].dt.day
    df_temp['year'] = df_temp['date'].dt.year
    df_temp['dayofweek'] = df_temp['date'].dt.dayofweek
    df_temp['quarter'] = df_temp['date'].dt.quarter
    df_temp['dayofmonth'] = df_temp['date'].dt.day
    #df_temp['weekofyear'] = df_temp['date'].dt.weekofyear
    df_temp['weekofyear'] = df_temp['date'].dt.isocalendar().week.astype(np.int64)
    df_temp['is_month_start'] = df_temp.date.dt.is_month_start.astype(np.int8)
    df_temp['is_month_end'] = df_temp.date.dt.is_month_end.astype(np.int8)
    df_temp['monday'] = df_temp.date.dt.weekday.eq(0).astype(np.uint8)
    df_temp['tuesday'] = df_temp.date.dt.weekday.eq(1).astype(np.uint8)
    df_temp['wednesday'] = df_temp.date.dt.weekday.eq(2).astype(np.uint8)
    df_temp['thursday'] = df_temp.date.dt.weekday.eq(3).astype(np.uint8)
    df_temp['friday'] = df_temp.date.dt.weekday.eq(4).astype(np.uint8)
    df_temp['saturday'] = df_temp.date.dt.weekday.eq(5).astype(np.uint8)
    df_temp['sunday'] = df_temp.date.dt.weekday.eq(6).astype(np.uint8)
    df_temp['is_quarter_end'] = df_temp['date'].dt.is_quarter_end.astype(np.uint8)
    df_temp['is_quarter_start'] = df_temp['date'].dt.is_quarter_start.astype(np.uint8)
    df_temp['is_year_end'] = df_temp['date'].dt.is_year_end.astype(np.uint8)
    df_temp['is_year_start'] = df_temp['date'].dt.is_year_start.astype(np.uint8)
    return df_temp

In [9]:
def fix_covid(df):
    #fixing covid data (multiplying values by a factor of 1.x) 
    
    df_copy = df.copy()
    df_copy = df_copy[df_copy['date'].between('2020-04-01', '2020-04-30')]
    df_copy['num_sold'] = df_copy['num_sold']*1.2
    df[df['date'].between('2020-04-01', '2020-04-30')] = df_copy
    
    df_copy = df.copy()
    df_copy = df_copy[df_copy['date'].between('2020-05-01', '2020-05-31')]
    df_copy['num_sold'] = df_copy['num_sold']*1.1
    df[df['date'].between('2020-05-01', '2020-05-31')] = df_copy
    
    return df

In [10]:
def GetHolidays(df):
    #dataset['date']=pd.to_datetime(dataset['date'])
    # Analyzing country specific holidays:-
    Holidays = pd.DataFrame(columns = ['date', 'holiday_name', 'country'])
    country_list = df.country.unique()
    min_year = df.year.min()
    max_year = df.year.max()
    number_of_days = (df['date'].max() - df['date'].min()).days + 1
    date_list = [df['date'].min() + timedelta(days=day) for day in range(number_of_days)]
    
    if min_year == max_year:
        years = [min_year]
    else:
        years = np.arange(min_year, max_year, 1)
        
    for country in tqdm(country_list):
        for h in CountryHoliday(country, years = years).items():   
            i=len(Holidays)
            Holidays.loc[i,'date']=h[0]
            Holidays.loc[i,'holiday_name']=h[1]
            Holidays.loc[i,'country']=country
    #print(Holidays)
    Holidays['isHoliday'] = 1

    # Merge on unique combinations of date and country
    date_country = df[['date', 'country']].drop_duplicates().reset_index(drop=True)
    date_country['date_str'] = date_country['date'].astype(str)
    Holidays['date_str'] = Holidays['date'].astype(str)
    date_country_holidays = pd.merge(date_country, Holidays[['date_str', 'country', 'isHoliday','holiday_name']], 
                                     how='left', on=['date_str', 'country'])

    # Merge back to the original DataFrame
    df = pd.merge(df, date_country_holidays[['date', 'country', 'isHoliday','holiday_name']], how='left', on=['date', 'country'])
    df['isHoliday'] = df['isHoliday'].fillna(0).astype(int)
    df['holiday_name'] = df['holiday_name'].fillna('Not Holiday')
    
    #Add Weekend
    #df['Friday']=(df['dayname']=='Friday').astype(int) 
    #df['Saturday']=(df['dayname']=='Saturday').astype(int) 
    #df['Sunday']=(df['dayname']=='Sunday').astype(int)  
    return df

In [11]:
def data_encoding(dataset,train=False):
    cat_features=['country','store','product']
    dataset=datetime_features(dataset)
    print("Date converted to Datetime features. Check returned dataframe if you want further details.")
    dataset=GetHolidays(dataset)
    print("Holidays Added")
    for feature in tqdm(cat_features):
        dictionary={}
        n=1
        print('feature:',feature)
        print('categories:',dataset[feature].unique())
        for sub_cat in dataset[feature].unique():
            dictionary[sub_cat]=n
            n+=1
        print('encoding rule:',dictionary)
        dataset[feature]=dataset[feature].map(dictionary)
        dummies=pd.get_dummies(dataset[feature],prefix=feature)
        for key in dummies:
            dataset[key]=dummies[key]
        dataset=dataset.drop(feature,axis=1)
    if train==True:
        dataset=fix_covid(dataset)
        print("Covid Years Data Normalised by a factor of 1.2.")
    dataset=dataset.drop(['date','holiday_name'],axis=1)
    return dataset

In [12]:
train=data_encoding(train,train=True)
test=data_encoding(test)

Date converted to Datetime features. Check returned dataframe if you want further details.


100%|██████████| 5/5 [00:00<00:00, 24.69it/s]


Holidays Added


100%|██████████| 3/3 [00:00<00:00, 15.82it/s]

feature: country
categories: ['Argentina' 'Canada' 'Estonia' 'Japan' 'Spain']
encoding rule: {'Argentina': 1, 'Canada': 2, 'Estonia': 3, 'Japan': 4, 'Spain': 5}
feature: store
categories: ['Kaggle Learn' 'Kaggle Store' 'Kagglazon']
encoding rule: {'Kaggle Learn': 1, 'Kaggle Store': 2, 'Kagglazon': 3}
feature: product
categories: ['Using LLMs to Improve Your Coding' 'Using LLMs to Train More LLMs'
 'Using LLMs to Win Friends and Influence People'
 'Using LLMs to Win More Kaggle Competitions' 'Using LLMs to Write Better']
encoding rule: {'Using LLMs to Improve Your Coding': 1, 'Using LLMs to Train More LLMs': 2, 'Using LLMs to Win Friends and Influence People': 3, 'Using LLMs to Win More Kaggle Competitions': 4, 'Using LLMs to Write Better': 5}


Covid Years Data Normalised by a factor of 1.2.
Date converted to Datetime features. Check returned dataframe if you want further details.


100%|██████████| 5/5 [00:00<00:00, 82.56it/s]


Holidays Added


100%|██████████| 3/3 [00:00<00:00, 61.03it/s]

feature: country
categories: ['Argentina' 'Canada' 'Estonia' 'Japan' 'Spain']
encoding rule: {'Argentina': 1, 'Canada': 2, 'Estonia': 3, 'Japan': 4, 'Spain': 5}
feature: store
categories: ['Kaggle Learn' 'Kaggle Store' 'Kagglazon']
encoding rule: {'Kaggle Learn': 1, 'Kaggle Store': 2, 'Kagglazon': 3}
feature: product
categories: ['Using LLMs to Improve Your Coding' 'Using LLMs to Train More LLMs'
 'Using LLMs to Win Friends and Influence People'
 'Using LLMs to Win More Kaggle Competitions' 'Using LLMs to Write Better']
encoding rule: {'Using LLMs to Improve Your Coding': 1, 'Using LLMs to Train More LLMs': 2, 'Using LLMs to Win Friends and Influence People': 3, 'Using LLMs to Win More Kaggle Competitions': 4, 'Using LLMs to Write Better': 5}


In [13]:
train.head()

,num_sold,month,day,year,dayofweek,quarter,dayofmonth,weekofyear,is_month_start,is_month_end,...,country_4,country_5,store_1,store_2,store_3,product_1,product_2,product_3,product_4,product_5
0,63.0,1,1,2017,6,1,1,52,1,0,...,0,0,1,0,0,1,0,0,0,0
1,66.0,1,1,2017,6,1,1,52,1,0,...,0,0,1,0,0,0,1,0,0,0
2,9.0,1,1,2017,6,1,1,52,1,0,...,0,0,1,0,0,0,0,1,0,0
3,59.0,1,1,2017,6,1,1,52,1,0,...,0,0,1,0,0,0,0,0,1,0
4,49.0,1,1,2017,6,1,1,52,1,0,...,0,0,1,0,0,0,0,0,0,1


In [14]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 136950 entries, 0 to 136949
Data columns (total 35 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   num_sold          136950 non-null  float64
 1   month             136950 non-null  int64  
 2   day               136950 non-null  int64  
 3   year              136950 non-null  int64  
 4   dayofweek         136950 non-null  int64  
 5   quarter           136950 non-null  int64  
 6   dayofmonth        136950 non-null  int64  
 7   weekofyear        136950 non-null  int64  
 8   is_month_start    136950 non-null  int8   
 9   is_month_end      136950 non-null  int8   
 10  monday            136950 non-null  uint8  
 11  tuesday           136950 non-null  uint8  
 12  wednesday         136950 non-null  uint8  
 13  thursday          136950 non-null  uint8  
 14  friday            136950 non-null  uint8  
 15  saturday          136950 non-null  uint8  
 16  sunday            13

In [15]:
X,y=train.drop("num_sold",axis=1),train["num_sold"]

In [16]:
#(tf.convert_to_tensor(X),tf.convert_to_tensor(y))

In [17]:
# cb_reg=CatBoostRegressor(loss_function='MAPE',verbose=3)
# cb_reg.fit(X,y)

In [18]:
# bag_reg=BaggingRegressor(cb_reg,n_estimators=5,n_jobs=-1,verbose=3)
# bag_reg.fit(X,y)

In [19]:
#bag_reg.predict(test)

In [20]:
#cb_reg.predict(test)

In [21]:
# rf_reg=RandomForestRegressor(n_estimators=1000,n_jobs=-1,verbose=3)
# rf_reg.fit(X,y)

In [22]:
def min_max_scaling(df):
    df_scaled = df.copy()
    for column in df_scaled.columns:
        df_scaled[column]=(df_scaled[column]-df_scaled[column].min())/(df_scaled[column].max()-df_scaled[column].min()+1e-16)    
    return(df_scaled)

In [23]:
X=min_max_scaling(X)
test=min_max_scaling(test)

In [24]:
X_train=X[:round(0.9*len(X))]
X_val=X[round(0.9*len(X)):]
y_train=y[:round(0.9*len(X))]
y_val=y[round(0.9*len(X)):]

In [25]:
X_train_tf=tf.convert_to_tensor(X_train)
X_val_tf=tf.convert_to_tensor(X_val)
y_train_tf=tf.convert_to_tensor(y_train.astype(np.float32))
y_val_tf=tf.convert_to_tensor(y_val.astype(np.float32))
test_tf=tf.convert_to_tensor(test)

In [26]:
test_tf

<tf.Tensor: shape=(27375, 34), dtype=float64, numpy=
array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       ...,
       [1., 1., 0., ..., 1., 0., 0.],
       [1., 1., 0., ..., 0., 1., 0.],
       [1., 1., 0., ..., 0., 0., 1.]])>

In [27]:
train_tf=tf.data.Dataset.from_tensor_slices((X_train_tf,y_train_tf))
val_tf=tf.data.Dataset.from_tensor_slices((X_val_tf,y_val_tf))

In [28]:
test_tf=tf.data.Dataset.from_tensor_slices(test_tf)

In [29]:
len(test_tf)

27375

In [30]:
#zeroes=tf.zeros(shape=(27375,),dtype=tf.dtypes.float32)

In [31]:
test_tf

<_TensorSliceDataset element_spec=TensorSpec(shape=(34,), dtype=tf.float64, name=None)>

In [32]:
strategy=tf.distribute.MirroredStrategy()

In [33]:
with strategy.scope():
    def smape_loss(y_true, y_pred):
        epsilon = 0.1
        summ = K.maximum(K.abs(y_true) + K.abs(y_pred) + epsilon, 0.5 + epsilon)
        smape = K.abs(y_pred - y_true) / summ * 2.0
        return smape

In [34]:
with strategy.scope():
    model=tf.keras.Sequential()
    model.add(tf.keras.layers.InputLayer(input_shape=(34,)))
#     model.add(tf.keras.layers.Dense(768,activation='relu',activity_regularizer=tf.keras.regularizers.l1_l2(0.2,0.15)))
#     model.add(tf.keras.layers.BatchNormalization())
#     model.add(tf.keras.layers.Dropout(0.9))
#     model.add(tf.keras.layers.Dense(2048,activation='relu',activity_regularizer=tf.keras.regularizers.l1_l2(0.01,0.15)))
#     model.add(tf.keras.layers.BatchNormalization())
#     model.add(tf.keras.layers.Dropout(0.9))
#     model.add(tf.keras.layers.Dense(630,activation='relu',activity_regularizer=tf.keras.regularizers.l1(0.9)))
#     model.add(tf.keras.layers.BatchNormalization())
#     model.add(tf.keras.layers.Dropout(0.9))
#     model.add(tf.keras.layers.Dense(512,activation='relu',activity_regularizer=tf.keras.regularizers.l1(0.1)))
#     model.add(tf.keras.layers.BatchNormalization())
#     model.add(tf.keras.layers.Dropout(0.1))
#     model.add(tf.keras.layers.Dense(512,activation='relu',activity_regularizer=tf.keras.regularizers.l1(0.1)))
#     model.add(tf.keras.layers.BatchNormalization())
#     model.add(tf.keras.layers.Dense(384,activation='relu',activity_regularizer=tf.keras.regularizers.l1(0.1)))
#     model.add(tf.keras.layers.BatchNormalization())
#     model.add(tf.keras.layers.Dense(384,activation='relu',activity_regularizer=tf.keras.regularizers.l1(0.1)))
#     model.add(tf.keras.layers.BatchNormalization())
#     model.add(tf.keras.layers.Dropout(0.3))
#     model.add(tf.keras.layers.Dense(384,activation='relu',activity_regularizer=tf.keras.regularizers.l1(0.1)))
#     model.add(tf.keras.layers.BatchNormalization())
#     model.add(tf.keras.layers.Dense(384,activation='relu',activity_regularizer=tf.keras.regularizers.l1(0.1)))
#     model.add(tf.keras.layers.BatchNormalization())
#     model.add(tf.keras.layers.Dense(256,activation='relu',activity_regularizer=tf.keras.regularizers.l1(0.1)))
#     model.add(tf.keras.layers.BatchNormalization())
#     model.add(tf.keras.layers.Dense(256,activation='relu',activity_regularizer=tf.keras.regularizers.l1(0.1)))
#     model.add(tf.keras.layers.BatchNormalization())
#     model.add(tf.keras.layers.Dense(256,activation='relu',activity_regularizer=tf.keras.regularizers.l1(0.1)))
#     model.add(tf.keras.layers.BatchNormalization())
#     model.add(tf.keras.layers.Dense(256,activation='relu',activity_regularizer=tf.keras.regularizers.l1(0.1)))
#     model.add(tf.keras.layers.BatchNormalization())
#     model.add(tf.keras.layers.Dropout(0.3))
#     model.add(tf.keras.layers.Dense(256,activation='relu',activity_regularizer=tf.keras.regularizers.l1(0.1)))
#     model.add(tf.keras.layers.BatchNormalization())
#     model.add(tf.keras.layers.Dense(256,activation='relu',activity_regularizer=tf.keras.regularizers.l1(0.1)))
#     model.add(tf.keras.layers.BatchNormalization())
#     model.add(tf.keras.layers.Dense(256,activation='relu',activity_regularizer=tf.keras.regularizers.l1(0.1)))
#     model.add(tf.keras.layers.BatchNormalization())
#     model.add(tf.keras.layers.Dense(128,activation='relu',activity_regularizer=tf.keras.regularizers.l1(0.1)))
#     model.add(tf.keras.layers.BatchNormalization())
#     model.add(tf.keras.layers.Dense(128,activation='relu',activity_regularizer=tf.keras.regularizers.l1(0.1)))
#     model.add(tf.keras.layers.BatchNormalization())
#     model.add(tf.keras.layers.Dense(128,activation='relu',activity_regularizer=tf.keras.regularizers.l1(0.1)))
#     model.add(tf.keras.layers.BatchNormalization())
#     model.add(tf.keras.layers.Dropout(0.3))
    model.add(tf.keras.layers.Dense(64,activation='relu',activity_regularizer=tf.keras.regularizers.l1(0.25)))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Dense(64,activation='relu',activity_regularizer=tf.keras.regularizers.l1(0.25)))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Dropout(0.3))
    model.add(tf.keras.layers.Dense(64,activation='relu',activity_regularizer=tf.keras.regularizers.l1(0.25)))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Dense(10,activation='relu',activity_regularizer=tf.keras.regularizers.l1(0.15)))
    model.add(tf.keras.layers.Dense(1,activation='linear',activity_regularizer=tf.keras.regularizers.l1(0.09)))
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),loss="mean_absolute_percentage_error",metrics=[smape_loss])

In [35]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                2240      
                                                                 
 batch_normalization (BatchN  (None, 64)               256       
 ormalization)                                                   
                                                                 
 dense_1 (Dense)             (None, 64)                4160      
                                                                 
 batch_normalization_1 (Batc  (None, 64)               256       
 hNormalization)                                                 
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense_2 (Dense)             (None, 64)                4

In [36]:
with strategy.scope():
    early_stopping=tf.keras.callbacks.EarlyStopping(monitor="val_loss",min_delta=0.01,patience=10,verbose=1,mode="min",
                                                    restore_best_weights=True,start_from_epoch=10)
    checkpoint=tf.keras.callbacks.ModelCheckpoint("/kaggle/working/mc_forecast_regressor2.h5",monitor="val_loss",
                                                  save_best_only=True,verbose=1,mode='min',save_freq='epoch')
    lr=tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss",factor=0.6,patience=3,verbose=3,mode="min",min_delta=0.01,
                                            cooldown=0,min_lr=0)

In [37]:
with strategy.scope():
    history=model.fit(train_tf.batch(96),epochs=300,callbacks=[early_stopping,checkpoint,lr],
              validation_data= val_tf.batch(96),use_multiprocessing=True)

Epoch 1/300
1283/1284 [============================>.] - ETA: 0s - loss: 60.7990 - smape_loss: 0.7748
Epoch 1: val_loss improved from inf to 50.46900, saving model to /kaggle/working/mc_forecast_regressor2.h5
1284/1284 [==============================] - 23s 12ms/step - loss: 60.7892 - smape_loss: 0.7746 - val_loss: 50.4690 - val_smape_loss: 0.5179 - lr: 0.0010
Epoch 2/300
1280/1284 [============================>.] - ETA: 0s - loss: 42.3404 - smape_loss: 0.3742
Epoch 2: val_loss improved from 50.46900 to 45.09184, saving model to /kaggle/working/mc_forecast_regressor2.h5
1284/1284 [==============================] - 15s 12ms/step - loss: 42.3352 - smape_loss: 0.3741 - val_loss: 45.0918 - val_smape_loss: 0.4119 - lr: 0.0010
Epoch 3/300
1283/1284 [============================>.] - ETA: 0s - loss: 38.4597 - smape_loss: 0.2929
Epoch 3: val_loss improved from 45.09184 to 41.64457, saving model to /kaggle/working/mc_forecast_regressor2.h5
1284/1284 [==============================] - 15s 12ms/s

In [38]:
with strategy.scope():
    model=keras.models.load_model("/kaggle/working/mc_forecast_regressor2.h5",compile=False)
    predictions=model.predict(test_tf.batch(len(test_tf)))

1/1 [==============================] - 0s 455ms/step


In [39]:
predictions

array([[  3.0122137],
       [  3.0122137],
       [  4.2719316],
       ...,
       [ 58.59351  ],
       [274.25464  ],
       [263.12646  ]], dtype=float32)

In [40]:
list(predictions.reshape(1,-1)[0])

[3.0122137,
 3.0122137,
 4.2719316,
 29.839863,
 14.566239,
 2.920184,
 3.0015965,
 8.085701,
 86.47618,
 33.788124,
 118.33578,
 58.88751,
 24.70649,
 117.0986,
 136.23788,
 119.163,
 54.51186,
 21.73066,
 162.381,
 105.63322,
 151.40985,
 129.74564,
 16.674812,
 140.88544,
 155.76172,
 3.0122137,
 3.0122137,
 975.91187,
 19.80326,
 31.006763,
 5.200351,
 1.8231721,
 7.8005185,
 5.0046377,
 36.655193,
 -0.4171281,
 -1.6811218,
 14.246342,
 -1.6365061,
 5.6701174,
 5.6978827,
 173.75407,
 4.5900555,
 -11.547377,
 8.630926,
 94.07515,
 3.0122137,
 22.456448,
 98.143654,
 89.16941,
 172.99458,
 167.26619,
 18.186047,
 101.25415,
 132.22057,
 3.0122137,
 3.0122137,
 125.83239,
 46.554173,
 163.99355,
 2.1630228,
 -0.37363577,
 17.378626,
 2.1132293,
 3.0122137,
 -7.7663984,
 -9.396149,
 21.371262,
 -4.0199943,
 2.83424,
 -22.398312,
 -20.48632,
 14.21493,
 -20.06491,
 -12.271296,
 30.22118,
 34.557686,
 4.0619884,
 25.02806,
 25.253231,
 88.559654,
 100.22834,
 7.3424983,
 59.290348,
 47.

In [41]:
submission=pd.DataFrame()
submission['id']=test.index+136950
num_sold=[round(x) for x in list(predictions.reshape(1,-1)[0])]
submission['num_sold']=num_sold
submission.set_index('id',inplace=True)
submission.to_csv("submission.csv",index=True)

In [42]:
submission

,num_sold
id,
136950,3
136951,3
136952,4
136953,30
136954,15
...,...
164320,343
164321,342
164322,59


In [43]:
pd.read_csv("/kaggle/working/submission.csv",index_col='id')

,num_sold
id,
136950,3
136951,3
136952,4
136953,30
136954,15
...,...
164320,343
164321,342
164322,59


In [44]:
len(submission)

27375

In [45]:
#ensemble or blenders can be a good idea?